### Problem 1
Read the noisy text data from the datafile typos20.data.
- Estimate output probabilities
- Estimate transition probabilities

Use Laplace smoothing.

Implement Viterbi algorithm for HMM. Find the most likely original text.

In [1]:
import numpy as np

In [2]:
orig = []
typo = []

c2i = {}

i = 1
with open('data_files/typos20.data') as fin:
    for line in fin:
        orig.append(line[0])
        if line[0] not in c2i:
            c2i[line[0]] = i
            i += 1
        typo.append(line[2])
        if line[2] not in c2i:
            c2i[line[2]] = i
            i += 1

In [6]:
i2c = {}
for c in c2i:
    i2c[c2i[c]] = c

In [3]:
N = len(c2i)

B = np.zeros((N+1, N+1))
B[1:, 1:] = 1

for i in range(len(orig)):
    i1 = c2i[orig[i]]
    i2 = c2i[typo[i]]
    B[i1, i2] += 1
for i in range(1, B.shape[0]):
    val = np.sum(B[i,:])
    for j in range(B.shape[1]):
        B[i,j] = B[i,j]/val

In [4]:
A = np.zeros((N+1, N+1))
A[0,1:] = 1.0/N
A[1:, 1:] = 1 # for Laplace smoothing

for i in range(1, len(orig)):
    i1 = c2i[orig[i-1]]
    i2 = c2i[orig[i]]
    A[i1, i2] += 1
for i in range(A.shape[0]):
    val = np.sum(A[i,:])
    for j in range(A.shape[1]):
        A[i,j] = A[i,j]/val

In [19]:
P = np.zeros((N+1, len(typo)+1))
S = np.zeros((N+1, len(typo)+1))
# P[i, j] = prob. to be in state i at time j
# S[i,j] = previous state
P[1:,0] = 1.0/N
S[1:,0] = 0

for i1, c in enumerate(typo):
    i = i1+1
    ic = c2i[c]
    for si in range(1, N+1):
        maxp = P[1,i-1] + np.log(A[1, si]) + np.log(B[si, ic])
        maxs = 1
        for sii in range(1, N+1):
            p = P[sii,i-1] + np.log(A[sii, si]) + np.log(B[si, ic])
            if p>maxp:
                maxp = p
                maxs = sii
        P[si, i] = maxp
        S[si, i] = maxs

In [37]:
# backwards tracing
i = P.shape[1]-1
s = np.argmax(P[1:,-1])


result = []
while i>0:
    result.append(i2c[s])
    s = int(S[s,i])
    i -= 1
result.reverse()

In [38]:
len(result)

181045

In [39]:
errors = 0
for i in range(len(orig)):
    if orig[i] != typo[i]:
        errors += 1
print(errors/len(orig))

errors = 0
for i in range(len(orig)):
    if orig[i] != result[i]:
        errors += 1
print(errors/len(orig))

0.16530144439227817
0.10481924383440581
